In [1]:
import urllib.request, urllib.error, urllib.parse
import os, time, re, json, requests, string, random, concurrent.futures, csv, warnings, nltk
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import plotly.graph_objects as go
from tqdm import tqdm
from PIL import Image
from ast import literal_eval
from pylab import plot, show
from bs4 import BeautifulSoup
from matplotlib.lines import Line2D
from scipy import stats
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds
from scipy.cluster.vq import kmeans, vq
import fastai
from fastai.collab import *
from fastai.tabular.all import *
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import neighbors, metrics, preprocessing
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from surprise.model_selection import cross_validate, train_test_split, KFold
from surprise import Reader, Dataset, SVD, KNNBasic, BaselineOnly, NormalPredictor, accuracy, dump
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\same\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\same\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## reading books

In [58]:
# dataset: https://www.kaggle.com/zygmunt/goodbooks-10k --> licence: CC BY-SA 4.0
books = pd.read_csv('datasets-books/dataset-goodbooks-10k/books.csv')

def prepare(x):
    x = str(x)
    x = x.replace("&", "and")
    return x.strip()

books["title"] = books["title"].apply(prepare)
books["original_title"] = books["original_title"].apply(prepare)

new_titles = []
for index_book, row_book in tqdm(books.iterrows()):
    if len(row_book['original_title']) < len(row_book['title']):
        new_titles.append(row_book['original_title'])
    else:
        new_titles.append(row_book['title'])

books["title"] = new_titles

books.head(2)

10000it [00:01, 9671.03it/s]


,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...


In [59]:
# dataset: https://www.kaggle.com/dylanjcastillo/7k-books-with-metadata --> license: CC0: Public Domain
books_7k = pd.read_csv("datasets-books/dataset-7k-books/books.csv")
books_7k = books_7k[["title", "description", "categories"]]
books_7k.head(2)

,title,description,categories
0,Gilead,A NOVEL THAT READERS and critics have been eag...,Fiction
1,Spider's Web,A new 'Christie for Christmas' -- a full-lengt...,Detective and mystery stories


In [60]:
# https://www.kaggle.com/jdobrow/57000-books-with-metadata-and-blurbs --> licence: ?
books_57k = pd.read_csv("datasets-books/dataset-57k-books/books_with_blurbs.csv")
books_57k = books_57k[["Title", "Blurb"]]
books_57k = books_57k.rename(columns={'Title': 'title', "Blurb": "description"})
books_57k.head(2)

,title,description
0,Decision in Normandy,"Here, for the first time in paperback, is an o..."
1,Flu: The Story of the Great Influenza Pandemic...,"The fascinating, true story of the world's dea..."


In [61]:
# dataset: https://www.kaggle.com/sp1thas/book-depository-dataset --> licence: CC BY-NC-SA 4.0
book_depository = pd.read_csv("datasets-books/dataset-book-depository/dataset.csv")
book_depository_categories = pd.read_csv("datasets-books/dataset-book-depository/categories.csv")
category_list = []
for book, row in tqdm(book_depository.iterrows()):
    c_list = literal_eval(row["categories"])
    c_book = []
    for c in c_list:
        get_c = book_depository_categories[book_depository_categories["category_id"] == c]["category_name"].item()
        c_book.append(get_c)
    category_list.append(c_book)

book_depository["categories"] = category_list
book_depository["categories"] = book_depository["categories"].apply(lambda x: ', '.join(x))
book_depository = book_depository[["title", "description", "categories"]]
book_depository.head(2)

56009it [01:56, 482.60it/s]


,title,description,categories
0,1984,Renowned urban artist Shepard Fairey's new loo...,"Contemporary Fiction, Classic Books & Novels, ..."
1,100 Deadly Skills: Survival Edition : The SEAL...,"These 100 skills, adapted for civilians from a...","Encyclopaedias & Reference Works, Reference Wo..."


In [62]:
# dataset: https://www.kaggle.com/meetnaren/goodreads-best-books --> licence: ?
books_2018_best = pd.read_csv("datasets-books/dataset-goodreads-2018-best-books/book_data.csv")
def prepare(x):
    x = str(x)
    x = x.replace('|', ", ")
    return x
books_2018_best["genres"] = books_2018_best["genres"].apply(prepare)
books_2018_best = books_2018_best[["book_title", "book_desc", "genres"]]
books_2018_best = books_2018_best.rename(columns={'book_title': 'title', "book_desc": "description", "genres": "categories"})
books_2018_best.head(2)

,title,description,categories
0,The Great Alone,"Alaska, 1974.Unpredictable. Unforgiving. Untam...","Fiction, Historical, Historical Fiction, Audio..."
1,Circe,"In the house of Helios, god of the sun and mig...","Fantasy, Fiction, Fantasy, Mythology, Historic..."


In [63]:
# dataset: https://www.kaggle.com/austinreese/goodreads-books --> licence: CC0: Public Domain
books_31_feat = pd.read_csv("datasets-books/dataset-goodreads-books-31-features/goodreads_books.csv")
def prepare(x):
    x = str(x)
    return re.sub('\s*[0-9]', '', x)
books_31_feat["genre_and_votes"] = books_31_feat["genre_and_votes"].apply(prepare)
books_31_feat = books_31_feat[["title", "description", "genre_and_votes"]]
books_31_feat = books_31_feat.rename(columns={"genre_and_votes": "categories"})
books_31_feat.head(2)

,title,description,categories
0,Inner Circle,Reed Brennan arrived at Easton Academy expecti...,"Young Adult, Mystery, Romance"
1,A Time to Embrace,"Ideje az Ã¶lelÃ©snek TÃ¶rtÃ©net a remÃ©nyrÅl,...","Christian Fiction, Christian, Fiction, Romance"


In [64]:
# dataset: https://www.kaggle.com/bilalyussef/google-books-dataset --> licence: google?
books_google = pd.read_csv("datasets-books/dataset-google-books/google_books_1299.csv")
books_google = books_google[["title", "description", "generes"]]
books_google = books_google.rename(columns={"generes": "categories"})
books_google.head(2)

,title,description,categories
0,Attack on Titan: Volume 13,NO SAFE PLACE LEFT At great cost to the Garris...,none
1,Antiques Roadkill: A Trash 'n' Treasures Mystery,Determined to make a new start in her quaint h...,"Fiction , Mystery &amp, Detective , Cozy , Gen..."


In [65]:
# dataset: https://www.kaggle.com/ymaricar/cmu-book-summary-dataset --> licence: CC BY-SA 3.0
books_cmu = pd.read_csv("datasets-books/dataset-cmu-book-summary/booksummaries.csv")
books_cmu = books_cmu[["BookTitle", "Summary", "GenreString"]]
books_cmu = books_cmu.rename(columns={'BookTitle': 'title', "Summary": "description", "GenreString": "categories"})
books_cmu.head(2)

,title,description,categories
0,Animal Farm,"Old Major, the old boar on the Manor Farm, ca...",Roman à clef Satire Children's literature Spec...
1,A Clockwork Orange,"Alex, a teenager living in near-future Englan...",Science Fiction Novella Speculative fiction Ut...


## merging

In [66]:
def count_nan():
    a = books_cmu["description"].isna().sum(), books_cmu["categories"].isna().sum()
    b = book_depository["description"].isna().sum(), book_depository["categories"].isna().sum()
    c = books_7k["description"].isna().sum(), books_7k["categories"].isna().sum()
    d = books_2018_best["description"].isna().sum(), books_2018_best["categories"].isna().sum()
    e = books_31_feat["description"].isna().sum(), books_31_feat["categories"].isna().sum()
    f = books_google["description"].isna().sum(), books_google["categories"].isna().sum()
    g = books_57k["description"].isna().sum(), books_57k.shape[0]
    print(a, b, c, d, e, f, g)

count_nan()
books_cmu = books_cmu.dropna()
book_depository = book_depository.dropna()
books_7k = books_7k.dropna()
books_2018_best = books_2018_best.dropna()
books_31_feat = books_31_feat.dropna()
books_google = books_google.dropna()
count_nan()

(0, 0) (4022, 0) (262, 99) (5, 0) (2575, 0) (3, 0) (0, 57510)
(0, 0) (0, 0) (0, 0) (0, 0) (0, 0) (0, 0) (0, 57510)


In [106]:
books_all = pd.concat([books_cmu, book_depository, books_7k, books_2018_best, books_31_feat, books_google, books_57k])
books_all

,title,description,categories
0,Animal Farm,"Old Major, the old boar on the Manor Farm, ca...",Roman à clef Satire Children's literature Spec...
1,A Clockwork Orange,"Alex, a teenager living in near-future Englan...",Science Fiction Novella Speculative fiction Ut...
2,The Plague,The text of The Plague is divided into five p...,Existentialism Fiction Absurdist fiction Novel
3,A Fire Upon the Deep,The novel posits that space around the Milky ...,Hard science fiction Science Fiction Speculati...
4,All Quiet on the Western Front,"The book tells the story of Paul Bäumer, a Ge...",War novel Roman à clef
...,...,...,...
57505,Tainted Trail,"Ukiah Oregon, half-man and half-alien raised b...",NaN
57506,Twelve Mile Limit,"On a Friday in early November, four people hea...",NaN
57507,The Man With the Red Tattoo (James Bond 007),On a quiet late-night flight from Tokyo to Lon...,NaN
57508,"Iron Fist (Star Wars: X-Wing Series, Book 6)",They are the Rebel Alliance's ultimate strike ...,NaN


## concat all

In [231]:
books = pd.read_csv('datasets-books/dataset-goodbooks-10k/books.csv')

def prepare(x):
    x = str(x)
    x = x.replace("&", "and")
    return x.strip()

books["title"] = books["title"].apply(prepare)
books["original_title"] = books["original_title"].apply(prepare)

new_titles = []
for index_book, row_book in tqdm(books.iterrows()):
    if len(row_book['original_title']) < len(row_book['title']):
        new_titles.append(row_book['original_title'])
    else:
        new_titles.append(row_book['title'])

books["title"] = new_titles

books.head(2)

books_summary_1 = books.merge(books_all, left_on='title', right_on='title', how='left')
books_summary_2 = books.merge(books_all, left_on='title', right_on='title', how='inner')

books_summary_1.head(2)

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,description,categories
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,The Hunger Games takes place in a nation know...,Science Fiction Adventure novel Alternate hist...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,"Before the start of the novel, Voldemort, con...",Children's literature Fantasy Speculative fict...


# remove chars

In [77]:
def non_ascii(x):
    if type(x) == str:
        x = x.replace('\"', '')
        x = x.replace('\\', '')
        x = x.replace("'", '')
        x = x.replace('\n', '')
        x = x.replace('\r', '')
        # x = x.replace('<', '')
        # x = x.replace('>', '')
        x = ''.join([i if ord(i) < 128 else ' ' for i in x])
    return x

def add_comma(x):
    x = str(x)
    if "," not in x:
        x = x.replace("fiction", "fiction,")
        x = x.replace("literature", "literature,")
        x = x.replace("Fiction", "Fiction,")
        x = x.replace("novel", "novel,")
        x = x.replace("book", "book,")
        x = x.replace("history", "history,")
    return x
        
books_summary_end["categories"] = books_summary_end["categories"].apply(add_comma)
books_summary_end = books_summary_end.applymap(non_ascii)

# crawler

In [ ]:
count1 = 0
write_html = False
skipped = []
skipped_descr = []
skipped_title = []
skipped_genre = []
for index, row in tqdm(books_summary_1.iterrows()):
    if pd.isnull(books_summary_1.iloc[index]["description"]) or pd.isnull(books_summary_1.iloc[index]["categories"]):
        url = "https://www.goodreads.com/book/show/" + str(row["book_id"])
        response = urllib.request.urlopen(url)
        webContent = response.read()
        if write_html:
            with open("htmls/" + str(index) + "-" + str(row["book_id"]) + ".html", 'wb') as f: # -> plus 2 seconds
                f.write(webContent)
                
        source = BeautifulSoup(webContent, 'html.parser')
        summary = source.find("div", attrs={"id": "description"})
        try:
            summary = summary.find_all("span")
        except:
            skipped.append((index, str(row["book_id"])))
        try:
            summary = summary[1]
        except:
            try:
                summary = summary[0]
            except:
                skipped_descr.append((index, str(row["book_id"])))
        try:
            books_summary_1.at[index, "description"] = summary.text
        except:
            skipped_descr.append((index, str(row["book_id"])))
        
        try:  
            book_title = source.find("h1", attrs={"id": "bookTitle"})
            books_summary_1.at[index, "title"] = book_title.text.strip()
        except:
            skipped_title.append((index, str(row["book_id"])))
        
        genre = source.find_all("a", attrs={"class": "actionLinkLite bookPageGenreLink"}, href=True)
        genres = ""
        try:
            genres += genre[0].text
        except:
            skipped_genre.append((index, str(row["book_id"])))
            pass
        try:
            genres += ", " + genre[1].text
        except:
            pass
        try:
            genres += ", " + genre[2].text
        except:
            pass
        if genres != "":
            books_summary_1.at[index, "categories"] = genres
        time.sleep(0.5)

print("continue:", count1)

In [167]:
def clean_data(x):
    x = str(x)
    word_tokens = word_tokenize(x)
    filtered_sentence = [w.lower() for w in word_tokens if w.isalpha()] # not w in stop_words and
    x =  " ".join(filtered_sentence)
    return str.lower(x.replace(" ", ""))

features = ['title', 'authors', 'description', 'categories']
filledna = books_summary_end[features]

for feature in features:
    filledna[feature] = filledna[feature].apply(clean_data)
    
filledna.head(2)

,title,authors,description,categories
0,thehungergames,suzannecollins,thehungergamestakesplaceinanationknownaspaneme...,sciencefictionadventurenovelalternatehistorysp...
1,harrypotterandthephilosophersstone,rowlingmarygrandpr,beforethestartofthenovelvoldemortconsideredthe...,childrensliteraturefantasyspeculativefictionfi...


In [168]:
def create_soup(x):
    return x['title'] + ' ' + x['authors'] + ' ' + x['description'] + ' ' + x['categories']

books_summary_end['soup'] = filledna.apply(create_soup, axis=1)
books_summary_end['soup']

0       thehungergames suzannecollins thehungergamesta...
1       harrypotterandthephilosophersstone rowlingmary...
2       twilight stepheniemeyer isabellabellaswanmoves...
3       tokillamockingbird harperlee thebookopenswitht...
4       thegreatgatsby scottfitzgerald thestorybeginsw...
                              ...                        
9995    bayoumoon ilonaandrews theedgeliesbetweenworld...
9996    meansofascent robertcaro robertcaroslifeoflynd...
9997    themauritiuscommand patrickobrian whenthenovel...
9998    cinderellaatemydaughterdispatchesfromthefrontl...
9999    thefirstworldwar johnkeegan thefirstworldwarcr...
Name: soup, Length: 10000, dtype: object

## popular

In [189]:
C = books_with_descr['average_rating'].mean()
m = books_with_descr['ratings_count'].quantile(0.9)
trending_books = books_with_descr.copy().loc[books_with_descr['ratings_count'] >= m]

def weighted_rating(x, m=m, C=C):
    v = x['ratings_count']
    R = x['average_rating']
    return (v / (v + m) * R) + (m / (m + v) * C)

trending_books['score'] = trending_books.apply(weighted_rating, axis=1)
trending_books = trending_books.sort_values('score', ascending=False)
trending_books[['title', 'ratings_count', 'average_rating', 'score']].head(10)

,title,ratings_count,average_rating,score
24,Harry Potter and the Deathly Hallows,1746574,4.61,4.578926
26,Harry Potter and the Half-Blood Prince,1678823,4.54,4.511454
17,Harry Potter and the Prisoner of Azkaban,1832823,4.53,4.504224
23,Harry Potter and the Goblet of Fire,1753043,4.53,4.503111
421,Harry Potter Series Box Set,190050,4.74,4.495659
134,A Storm of Swords,469022,4.54,4.450127
191,The Name of the Wind,400101,4.55,4.445690
20,Harry Potter and the Order of the Phoenix,1735368,4.46,4.436452
1,Harry Potter and the Philosophers Stone,4602479,4.44,4.431228
160,The Return of the King,463959,4.51,4.424371


# reading the movie dataset

In [80]:
# dataset: https://www.kaggle.com/rounakbanik/the-movies-dataset --> licence: CC0: Public Domain
rating_small = pd.read_csv("datasets-movies/dataset-the-movie/ratings_small.csv")
print(rating_small.shape)
rating_small.head(2)

(100004, 4)


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [81]:
rating_large = pd.read_csv("datasets-movies/dataset-the-movie/ratings.csv")
print(rating_large.shape)
rating_large.head(2)

(26024289, 4)


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435


In [270]:
movies = pd.read_csv("datasets-movies/dataset-the-movie/movies_metadata.csv")
# img url: https://image.tmdb.org/t/p/original/ + poster_path
# movies = movies.fillna("")

wrong_id = []
counter = 0
for i in movies['id']:
    try:
        i = int(i)
    except:
        wrong_id.append(counter)
    counter += 1
movies = movies.drop([19730, 29503, 35587]) # [19730, 29503, 35587]

def clean_imdb_id(x):
    if type(x) == str:
        x = x.split("tt")[1]
    try:
        x = int(x)
    except:
        x = -1
    return x
movies["clean_imdb_id"] = movies["imdb_id"].apply(clean_imdb_id)

print(movies.shape)
movies.head(2)

(45463, 25)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,clean_imdb_id
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,114709
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,113497


In [218]:
credits = pd.read_csv("datasets-movies/dataset-the-movie/credits.csv")
print(credits.shape)
credits.head(2)

(45476, 3)


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [154]:
keywords = pd.read_csv("datasets-movies/dataset-the-movie/keywords.csv")
print(keywords.shape)
keywords.head(2)

(46419, 2)


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [155]:
links_small = pd.read_csv("datasets-movies/dataset-the-movie/links_small.csv")
print(links_small.shape)
links_small.head(2)

(9125, 3)


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [156]:
links_large = pd.read_csv("datasets-movies/dataset-the-movie/links.csv")
print(links_large.shape)
links_large.head(2)

(45843, 3)


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [157]:
# dataset: https://www.kaggle.com/neha1703/movie-genre-from-its-poster --> licence: ?
posters = pd.read_csv("datasets-movies/dataset-posters/imdbid_posterurl.csv", usecols=["imdbId", "Poster"])
posters = posters.dropna()
posters.head(2)

,imdbId,Poster
0,114709,https://images-na.ssl-images-amazon.com/images...
1,113497,https://images-na.ssl-images-amazon.com/images...


In [203]:
movie_links_with_poster = links_large.merge(posters, left_on="imdbId", right_on="imdbId", how="left")
movie_links_with_poster = movie_links_with_poster.rename(columns={'imdbId': 'clean_imdb_id', 'Poster': 'poster'})
movie_links_with_poster = movie_links_with_poster[["clean_imdb_id", "poster"]]
movie_links_with_poster["clean_imdb_id"] = movie_links_with_poster["clean_imdb_id"].astype('int')
print(movie_links_with_poster.shape)
movie_links_with_poster.head(2)

(46435, 2)


,clean_imdb_id,poster
0,114709,https://images-na.ssl-images-amazon.com/images...
1,113497,https://images-na.ssl-images-amazon.com/images...


In [271]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
movies['id'] = movies['id'].astype('int')

movies = pd.merge(movies, credits, on='id', how="left")
movies = pd.merge(movies, keywords, on='id', how="left")
movies = pd.merge(movies, movie_links_with_poster, on='clean_imdb_id', how="left")

print(movies.shape)
movies.head(2)

(47220, 29)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,clean_imdb_id,cast,crew,keywords,poster
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,114709,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",https://images-na.ssl-images-amazon.com/images...
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,113497,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",https://images-na.ssl-images-amazon.com/images...


# clearing

In [275]:
# genres, cast, crew, keywords, spoken_languages, production_countries, production_companies, belongs_to_collection
def get_clean(x):
    if x == np.nan or x == '' or x == 'nan':
        return ''
    try:
        x = literal_eval(x)
    except:
        print(x)
    cleaned = []
    if isinstance(x, list):
        if len(x) >= 3:
            for i in x[:3]:
                cleaned.append(i["name"])
            return ", ".join(cleaned)
        else:
            for i in x:
                cleaned.append(i["name"])
            return ", ".join(cleaned)
    return ''

def get_director(x):
    if x == np.nan or x == '' or x == 'nan':
        return ''
    x = literal_eval(x)
    for i in x:
        if i["job"] == "Director":
            return i["name"]
    return ''

def get_collection(x):
    if x == np.nan or x == '' or x == 'nan':
        return ''
    x = literal_eval(x)
    return x["name"]

movies['clean_genres'] = movies['genres'].apply(get_clean)
movies['clean_cast'] = movies['cast'].apply(get_clean)
movies['director'] = movies['crew'].apply(get_director)
movies['clean_keywords'] = movies['keywords'].apply(get_clean)
movies['clean_spoken_languages'] = movies['spoken_languages'].apply(get_clean)
movies['clean_production_countries'] = movies['production_countries'].apply(get_clean)
movies['clean_production_companies'] = movies['production_companies'].apply(get_clean)
movies['clean_belongs_to_collection'] = movies['belongs_to_collection'].apply(get_collection)

movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,clean_imdb_id,cast,crew,keywords,poster,clean_genres,clean_cast,director,clean_keywords,clean_spoken_languages,clean_production_countries,clean_production_companies,clean_belongs_to_collection
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,3.73554e+08,81,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415,114709,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",https://images-na.ssl-images-amazon.com/images...,"Animation, Comedy, Family","Tom Hanks, Tim Allen, Don Rickles",John Lasseter,"jealousy, toy, boy",English,United States of America,Pixar Animation Studios,Toy Story Collection
1,False,,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,2.62797e+08,104,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413,113497,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",https://images-na.ssl-images-amazon.com/images...,"Adventure, Fantasy, Family","Robin Williams, Jonathan Hyde, Kirsten Dunst",Joe Johnston,"board game, disappearance, based on children's...","English, Français",United States of America,"TriStar Pictures, Teitler Film, Interscope Com...",


# user based recommender

In [ ]:
reader = Reader()
ratings = pd.read_csv('datasets-movies/dataset-the-movie/ratings_small.csv')

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=5)
svd = SVD()

for trainset, testset in kf.split(data):
    svd.fit(trainset)
    predictions = svd.test(testset)
    accuracy.rmse(predictions, verbose=True)

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
svd.predict(1, 20, 3) # how much would rate user 1 to movie 20

# fastai
source: https://github.com/fastai/fastbook/blob/master/08_collab.ipynb

# fastai - books

In [2]:
book_dataset = pd.read_csv("datasets-site/books_dataset.csv", usecols=["id", "title"])
book_dataset = book_dataset.rename(columns={"id": "book_id"})
print(book_dataset.shape)
book_dataset.head(2)

book_ratings = pd.read_csv("datasets-site/books_ratings.csv")
book_ratings = book_ratings[["user_id", "book_id", "rating"]]
book_ratings = book_ratings.merge(book_dataset)
print(book_ratings.shape)
book_ratings.head(2)

book_ratings = pd.read_csv("books_ratings.csv")
print(book_ratings.shape)
book_ratings.head(2)

(12698, 4)


,user_id,book_id,rating,title
0,1,7331,5,Wicca: A Guide for the Solitary Practitioner
1,1,7861,5,The Magic Mirror of M.C. Escher


In [146]:
book_dls = CollabDataLoaders.from_df(book_ratings, item_name='title', seed=1)
book_learn = collab_learner(book_dls, y_range=(0.5, 5.5))
book_learn.fine_tune(6, wd=0.1)

epoch,train_loss,valid_loss,time
0,1.598441,1.564797,00:01


epoch,train_loss,valid_loss,time
0,1.499040,1.537886,00:01
1,1.406583,1.472987,00:01
2,1.281187,1.410674,00:01
3,1.106393,1.369578,00:01
4,1.005396,1.351762,00:01
5,0.969777,1.348881,00:01


In [145]:
books_dls = CollabDataLoaders.from_df(book_ratings, item_name='title', seed=1)
books_collab_filtering = collab_learner(books_dls, y_range=(0.5, 5.5))
books_collab_filtering.model_dir = "."
books_collab_filtering = books_collab_filtering.load("books_collab_filtering")

book_learn.show_results(max_n=3)

,user_id,title,rating,rating_pred
0,174,146,4,3.514095
1,460,1923,4,3.191102
2,213,2282,3,3.321907


In [166]:
book_learn.model_dir = "."
book_learn.save("books_collab_filtering")
# book_learn = book_learn.load("books_collab_filtering")

book_user_embd = book_learn.model.u_weight.weight
book_user_bais = book_learn.model.u_bias.weight
book_book_embd = book_learn.model.i_weight.weight
book_book_bais = book_learn.model.i_bias.weight

book_dls.classes

{'user_id': (#666) ['#na#',1,2,3,4,5,6,7,8,9...],
 'title': (#5988) ['#na#','             ','       1 [Kisu Yorimo Hayaku 1]','#Girlboss','...And Ladies of the Club','1,000 Places to See Before You Die','1/4 Gram','10% Happier: How I Tamed the Voice in My Head, Reduced Stress Without Losing My Edge, and Found Self-Help That Actually Works','100 Bullets, Vol. 1: First Shot, Last Call','100 Love Sonnets'...]}

In [61]:
book_learn.get_preds(dl=[(tensor([[269, 4839], [217, 2498]], device='cuda:0'), tensor([[4], [4]], device='cuda:0'))])

(tensor([3.0445, 3.1135]),
 tensor([[4],
         [4]]))

In [133]:
def item_similarities(title, dls, emdb):
    book_index = dls.classes["title"].o2i[title]
    distance = nn.CosineSimilarity(dim=1)(emdb, emdb[book_index][None])
    index = distance.argsort(descending=True)[1:6]
    return list(dls.classes["title"][index])

item_similarities("The Hunger Games", book_dls, book_book_embd)

['Harry Potter and the Deathly Hallows',
 'Divergent',
 'To Kill a Mockingbird',
 'The Great Gatsby',
 'Twilight']

In [134]:
def user_user_similarities(user_id, embd, dls):
    user_index = movie_dls.classes["user_id"].o2i[user_id]
    distance = nn.CosineSimilarity(dim=1)(embd, embd[user_index][None])
    index = distance.argsort(descending=True)[1:6]
    return list(dls.classes["user_id"][index])

user_user_similarities(12, book_user_embd, book_dls)

[392, 445, 160, 521, 33]

In [149]:
book_dls.items.head(1)

,user_id,book_id,rating,title
11598,536,7625,4,554


In [136]:
def get_user_not_rated(user_id, ratings):
    items = set(ratings["title"].unique()) - set(ratings[ratings["user_id"] == user_id]["title"].unique())
    u_m = list(zip([user_id] * len(items), items))
    return torch.tensor(u_m, device="cuda:0"), torch.zeros(len(items), 1, device="cuda:0")

In [137]:
def rec_item_for_user(user_id, learned, ratings, dataset, search="book_id"):
    user_item, user_rate = get_user_not_rated(user_id, ratings)
    preds, _ = learned.get_preds(dl=[(user_item, user_rate)])
    title_preds = list(zip(user_item[:, 1].tolist(), preds.tolist()))
    title_preds = sorted(title_preds, key=lambda x: x[1], reverse=True)
    found = [title_id[0] for title_id in title_preds[:5]]
    found = ratings[ratings["title"].isin(found)][search].unique()
    if search == "book_id":
        search = "id"
    return dataset[dataset[search].isin(found)]["title"]

rec_item_for_user(1, book_learn, book_dls.items, book_dataset)

2105                                                       This Man Confessed
2464                                                   Stumbling on Happiness
4994                                                             Patient Zero
7442    Short Stories from Hogwarts of Power, Politics and Pesky Poltergeists
7522                             Hogwarts: An Incomplete and Unreliable Guide
Name: title, dtype: object

# fastai - movies

In [99]:
movie_dataset = pd.read_csv("datasets-site/movies_dataset.csv")
print(movie_dataset.shape)
movie_dataset.head(2)

movie_ratings = pd.read_csv("datasets-movies/dataset-the-movie/ratings_small.csv")
movie_ratings = movie_ratings.rename(columns={'userId': 'user_id', 'movieId': 'movie_id'})
print(movie_ratings.shape)
movie_ratings.head(2)

links = pd.read_csv("datasets-movies/dataset-the-movie/links.csv", usecols=["movieId", "imdbId"])
links = links.rename(columns={'movieId': 'movie_id', 'imdbId': 'imdb_id'})

def clean_imdb_id(x):
    x = str(x)
    x = "tt" + "0" * (7 - len(x)) + x
    return x
links["imdb_id"] = links["imdb_id"].apply(clean_imdb_id)

print(links.shape)
links.head(2)

(45843, 2)


,movie_id,imdb_id
0,1,tt0114709
1,2,tt0113497


In [27]:
movie_dataset = movie_dataset.merge(links)
print(movie_dataset.shape)
movie_dataset.to_csv("movie_dataset_with_id.csv", index=False)

movie_ratings = pd.read_csv("movies_ratings.csv")
print(movie_ratings.shape)
movie_ratings.head(2)

(99765, 5)


,user_id,movie_id,rating,timestamp,title
0,1,31,2.5,1260759144,Dangerous Minds
1,7,31,3.0,851868750,Dangerous Minds


In [28]:
movie_dls = CollabDataLoaders.from_df(movie_ratings, item_name='title', seed=1)
movie_learn = collab_learner(movie_dls, y_range=(0.5, 5.5))
movie_learn.fine_tune(6, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.913022,0.885270,00:13


epoch,train_loss,valid_loss,time
0,0.795705,0.836336,00:12
1,0.751556,0.792403,00:13
2,0.683684,0.775082,00:13
3,0.624419,0.767629,00:13
4,0.592811,0.765430,00:14
5,0.571576,0.765258,00:14


In [144]:
movie_user_embd = movie_learn.model.u_weight.weight
movie_user_bais = movie_learn.model.u_bias.weight
movie_movie_embd = movie_learn.model.i_weight.weight
movie_movie_bais = movie_learn.model.i_bias.weight

movie_learn.show_results(max_n=3)

,user_id,title,rating,rating_pred
0,4.0,2639.0,4.0,3.990846
1,178.0,8405.0,3.5,3.436476
2,386.0,5826.0,1.0,2.451004


In [138]:
item_similarities("Pulp Fiction", movie_dls, movie_movie_embd)

['The Wrestler',
 'The Good, the Bad and the Ugly',
 'Princess Mononoke',
 'Voices from the List',
 'Three Colors: White']

In [139]:
user_user_similarities(12, movie_user_embd, movie_dls)

[84, 553, 308, 325, 418]

In [140]:
rec_item_for_user(1, movie_learn, movie_dls.items, movie_dataset, search="movie_id")

178     Mighty Morphin Power Rangers: The Movie
414                     The Beverly Hillbillies
1337                                   Jaws 3-D
1433                                   Anaconda
3463                          Battlefield Earth
Name: title, dtype: object

# pca - books

In [168]:
def dataset_pca(item_ratings, learned, embd):
    g = item_ratings.groupby('title')['rating'].count()
    top_items = g.sort_values(ascending=False).index.values[:1000]
    top_idxs = tensor([learned.dls.classes['title'].o2i[i] for i in top_items])
    item_w = embd[top_idxs].cpu().detach()
    item_pca = item_w.pca(3)
    fac0, fac1, fac2 = item_pca.t()
    idxs = np.random.choice(len(top_items), 50, replace=False)
    idxs = list(range(50))
    X = fac0[idxs]
    Y = fac2[idxs]
    plt.figure(figsize=(12, 12))
    plt.scatter(X, Y)
    for i, x, y in zip(top_items[idxs], X, Y):
        plt.text(x, y, i, color=np.random.rand(3) * 0.7, fontsize=11)
    plt.show()
    
dataset_pca(book_ratings, book_learn, book_book_embd)

# tfidf - books

In [ ]:
book_dataset = pd.read_csv("datasets-site/books_dataset.csv")
print(book_dataset.shape)
book_dataset.head(1)

features = ["authors", "title", "clean_description", "categories"]
for feature in features:
    book_dataset[feature] = book_dataset[feature].apply(lambda x: str(x))

def create_soup(x):
    return x['authors'] + ' ' + x['title'] + ' ' + x['clean_description'] + ' ' + x['categories']

book_dataset['soup'] = book_dataset.apply(create_soup, axis=1)
book_dataset['soup'][0]

In [62]:
book_tf = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), min_df=0, stop_words='english')
book_tfidf_matrix = book_tf.fit_transform(book_dataset['soup'])
book_cosine = cosine_similarity(book_tfidf_matrix, book_tfidf_matrix)

In [96]:
def get_recommendations(title, dataset, cosine):
    item_index = dataset[dataset["title"] == title].index[0]
    similar_items = list(enumerate(cosine[item_index]))
    sorted_items = sorted(similar_items, key=lambda x: x[1], reverse=True)
    print("{:>1} {:>40} \t {:>15}".format("Index:", "Title:", "Similarity:"))
    for i, item in enumerate(sorted_items[1:6]):
        title = dataset.iloc[item[0]]["title"]
        similarity = item[1]
        print("{:>5}: {:>40} \t {:>15.4f}".format(i + 1, title, similarity))

In [97]:
get_recommendations("Harry Potter and the Deathly Hallows", book_dataset, book_cosine)

Index:                                   Title: 	     Similarity:
    1: Harry Potter and the Order of the Phoenix 	          0.4955
    2:  Harry Potter and the Philosophers Stone 	          0.4677
    3:      Harry Potter and the Goblet of Fire 	          0.4570
    4:   Harry Potter and the Half-Blood Prince 	          0.4432
    5: Harry Potter and the Prisoner of Azkaban 	          0.3690


# tfidf - movies

In [ ]:
movie_dataset = pd.read_csv("datasets-site/movies_dataset.csv")
movie_dataset = movie_dataset.fillna('')
print(movie_dataset.shape) # (45433, 7)
movie_dataset.head(1)

In [72]:
stop_words = set(stopwords.words('english'))
def clean_data(x):
    x = str(x)
    word_tokens = word_tokenize(x)
    filtered_sentence = [w.lower() for w in word_tokens if w.isalpha() and w.lower() not in stop_words]
    return " ".join(filtered_sentence)

features = ["overview", "keywords", "tagline"]
for feature in features:
    movie_dataset[feature] = movie_dataset[feature].apply(clean_data)

def create_soup(x):
    return x['director'].lower() + ' ' + x['title'].lower() + ' ' + x['overview'] + ' ' + x['genres'].lower() + \
            ' ' + x['cast'].lower() + ' ' + x['keywords'] + ' ' + x['tagline']

movie_dataset['soup'] = movie_dataset.apply(create_soup, axis=1)
movie_dataset['soup'][0]

'john lasseter toy story led woody andys toys live happily room andys birthday brings buzz lightyear onto scene afraid losing place andys heart woody plots buzz circumstances separate buzz woody owner duo eventually learns put aside differences animation, comedy, family tom hanks, tim allen, don rickles jealousy toy boy '

In [76]:
movie_tf = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), min_df=0, stop_words='english')
movie_tfidf_matrix = movie_tf.fit_transform(movie_dataset['soup'][:10000])
movie_cosine = cosine_similarity(movie_tfidf_matrix, movie_tfidf_matrix)

In [98]:
get_recommendations("The Shawshank Redemption", movie_dataset, movie_cosine)

Index:                                   Title: 	     Similarity:
    1:                                   Prison 	          0.0621
    2:                                 Brubaker 	          0.0620
    3:                              Bob Roberts 	          0.0570
    4:                          Johnny Handsome 	          0.0547
    5:                 Night Falls on Manhattan 	          0.0530
